### 문맥과 기억: AI를 더 현실적으로 만들기

### 학습 내용
 * No Context = 무질서한 무작위성(Chaos of randomness)  
 * History=Context - 이전의 요청과 응답을 사용하기
 * 새로운 접근 방식 - LIFO(Last in First out)
 * 선택적인 문맥 - LIFO 방식의 한계에 대한 제안

### 사전 준비
 * 구글 코랩 환경은 일정 시간이후에 초기화가 되기 때문에 두가지 작업을 매번 수행해야 함.
   * chatgpt.env 파일 생성이 필요.
     * 준비된 chatgpt.env를 내용을 변경하여 업로드 하거나 또는 API_KEY와 ORG_ID를 확인하여 생성한다.
   * pip install openai 설치

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.2 MB/s eta 0:00:00


### No Context = 무질서한 무작위성
  * 우리가 만든 대화 에이전트가 기본적으로 메모리를 가지고 있지 않음.

In [ ]:
import openai
import os

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value

    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

init_api()

initial_prompt = """You: Hi there!
You: Hello!
AI: How are you?
You: {}
AI: """

while True :
    prompt = input("You(q:quit): ")
    if prompt=='q':
        break
    response = openai.Completion.create(
        engine = "text-davinci-003",
        prompt = initial_prompt.format(prompt),
        temperature = 1,
        max_tokens = 100,
        stop = [" You:", " AI:"]
    )
    print("AI:", response.choices[0].text.strip())


You(q:quit): 안녕
AI: 하이! 너 어떻게 지내?
You(q:quit): 'blah'라는 문자열을 기억하렴. 나는 나중에 이 단어를 물어볼거야.
AI: 기억했습니다! 나중에 다시 묻습니다.
You(q:quit): 방금 너가 기억한 단어는 뭐니?
AI: 기억한 단어가 없습니다. 무엇을 도와드릴까요?
You(q:quit): 내가 너한테 기억하라고 한 단어 기억안나니?
AI: 그 단어는 무엇입니까?
You(q:quit): 나는 너한테 방금전에 알려주었는데…
AI: 알겠습니다. 당신은 어떻게 지내고 계십니까?
You(q:quit): q


### History=Context - 이전의 요청과 응답을 사용하기

#### 이전의 요청과 답변을 변수에 기억시킴.

In [ ]:
import openai
import os

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value

    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

init_api()

initial_prompt = """You: Hi there!
You: Hello!
AI: How are you?
You: {}
AI: """

history = ""

while True :
    prompt = input("You(q:quit): ")
    if prompt=='q':
        break
    response = openai.Completion.create(
        engine = "text-davinci-003",
        prompt = initial_prompt.format(history + prompt),
        temperature = 1,
        max_tokens = 100,
        stop = [" You:", " AI:"],
    )
    response_text = response.choices[0].text
    history += "You: " + prompt + " \n " + "AI: " + response_text + " \n "
    print("AI: " + response_text)


You(q:quit): 안녕
AI:  안녕하세요! 만나서 반갑습니다! 무엇을 도와드릴까요?
You(q:quit): 'blah'라는 문자열을 기억하렴. 나는 나중에 이 단어를 물어볼거야.
AI: 그렇군요! 'blah'을 기억하겠습니다. 다른 도움이 필요하신가요?
You(q:quit): 방금 너가 기억한 단어는 뭐니?
AI: 기억한 단어는 'blah'입니다. 다른 도움이 필요하신가요?
You(q:quit): q


### 새로운 접근 방식 - LIFO(Last in First out)

* 이 접근 방식의 아이디어
  * 사용자는 항상 문맥을 가지고 대화 시작
  * 문맥은 변화고, 대화도 변한다.
  * 사용자는 최근 2-5개 프롬프트에 문맥을 포함시킬 가능성이 매우 크다.
* 히스토리를 저장할 텍스트 파일을 만들고, 구분자로 프롬프트와 답변을 저장한다.


In [ ]:
import openai
import os

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value

    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

def save_history_to_file(history):
    with open("history.txt", "w+") as f:
        f.write(history)

def load_history_from_file():
    with open("history.txt", "r") as f:
        return f.read()

def get_relevant_history(history):
    history_list = history.split(separator)
    if len(history_list) > 2:
        return separator.join(history_list[-2:])
    else:
        return history

init_api()

initial_prompt = """You: Hi there!
You: Hello!
AI: How are you?
You: {}
AI: """

history = ""
relevant_history = ""
separator = "#####"

while True :
    prompt = input("You(q:quit): ")
    if prompt=='q':
        break
    relevant_history = get_relevant_history(load_history_from_file())
    response = openai.Completion.create(
        engine = "text-davinci-003",
        prompt = initial_prompt.format(relevant_history + prompt),
        temperature = 1,
        max_tokens = 100,
        stop = [" You:", " AI:"],
    )
    response_text = response.choices[0].text
    history += "\nYou: " + prompt + "\nAI: " + response_text + "\n" + separator
    save_history_to_file(history)
    print("AI: " + response_text)


You(q:quit): 안녕
AI:  환영합니다! 무엇을 도와드릴까요?
You(q:quit): 'blah'라는 문자열을 기억하렴. 나는 나중에 이 단어를 물어볼거야.
AI: 기억하겠습니다. 'blah'라고 하셨습니다.
You(q:quit): 방금 너가 기억한 단어는 뭐니?
AI: 기억한 단어는 'blah'입니다.
You(q:quit): 'blah11'라는 문자열을 기억하렴. 나는 나중에 이 단어를 물어볼거야.
AI: 기억한 단어는 'blah11'입니다.
You(q:quit): 'blah22'라는 문자열을 기억하렴. 나는 나중에 이 단어를 물어볼거야.
AI: 기억한 단어는 'blah22'입니다.
You(q:quit): 'blah33'라는 문자열을 기억하렴. 나는 나중에 이 단어를 물어볼거야.
AI: 기억한 단어는 'blah33'입니다.
You(q:quit): 너가 지금까지 기억한 모든 단어를 알려줄 수 있니?
AI: 현재까지 기억한 모든 단어는 'blah33' 하나뿐입니다.
You(q:quit): q


### 선택적인 문맥 - LIFO의 한계의 해결 제안

* 초기 프롬프트가 텍스트 파일에 저장
* 사용자가 프롬프트에 입력
* 모든 상호 작용에 대한 임베딩 생성
* 임베딩과 코사인 유사도를 활용
* 가장 높은 n개의 내용을 사용자에게 프롬프트와 함께 보내기

#### 사전 다운로드 및 설치

In [ ]:
!python -m spacy download en_core_web_md

2023-08-14 09:47:12.706563: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 09:47:13.885317: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


#### 모든 코드를 합친 코드

In [ ]:
import openai
import os
import spacy
import numpy as np

# 사전 학습된 spaCy 모델을 로드합니다.
# en_core_web_md는 spaCy 라이브러리에서 제공하는 영어 언어 모델입니다.
# 텍스트 처리와 자연어 이해 작업 수행(토큰화, 품사태깅, 개체명 인식 등에 사용)
nlp = spacy.load('en_core_web_md')

def init_api():
    with open("chatgpt.env") as env:
        for line in env:
            key, value = line.strip().split("=")
            os.environ[key] = value
    openai.api_key = os.environ.get("API_KEY")
    openai.organization = os.environ.get("ORG_ID")

def save_history_to_file(history):
    """ 대화 기록을 파일에 저장합니다. """
    with open("history.txt", "w+") as f:
        f.write(history)

def load_history_from_file():
    """ 파일로부터 모든 대화 기록을 불러옵니다. """
    with open("history.txt", "r") as f:
        return f.read()

def cos_sim(a, b):
    """ 두 문자열 사이의 코사인 유사도를 계산합니다.
    사용자 입력과 기록된 대화 세그먼트 사이의 유사성 비교에 사용됩니다.
    """
    a = nlp(a)
    a_without_stopwords = nlp(' '.join([t.text for t in a if not t.is_stop]))
    b = nlp(b)
    b_without_stopwords = nlp(' '.join([t.text for t in b if not t.is_stop]))
    return a_without_stopwords.similarity(b_without_stopwords)

def sort_history(history, user_input):
    """ 사용자 입력과 대화 기록 세그먼트 사이의 코사인 유사도에 기반하여 대화 기록을
정렬합니다. History는 세팅자(separator)로 구분된 세그먼트들의 문자열입니다.
    """
    segments = history.split(separator)
    similarities = []
    for segment in segments:
        # 사용자 입력과 세그먼트 간의 코사인 유사도를 얻습니다.
        similarity = cos_sim(user_input, segment)
        similarities.append(similarity)
    sorted_similarities = np.argsort(similarities)
    sorted_history = ""
    for i in range(1, len(segments)):
        sorted_history += segments[sorted_similarities[i]] + separator
    save_history_to_file(sorted_history)

def get_latest_n_from_history(history, n):
    """
    대화 기록에서 최근 n개의 세그먼트를 가져옵니다.
    History는 세팅자(separator)로 구분된 세그먼트들의 문자열입니다.
    """
    segments = history.split(separator)
    return separator.join(segments[-n:])

initial_prompt_1 = """You: Hi there!
AI: Hello! #####
You: How are you?
AI: I am fine, thank you. #####
You: Do you know cars?
AI: Yes, I have some knowledge about cars. #####
You: Do you eat Pizza?
AI: I don't eat pizza. I am an AI that is not able to eat. #####
You: Have you ever been to the moon?
AI: I have never been to the moon. What about you? #####
You: What is your name?
AI: My name is Pixel. What is your name? #####
You: What is your favorite movie?
AI: My favorite movie is The Matrix. Follow the white rabbit :) #####
"""

initial_prompt_2 = """You: {}
AI: """

initial_prompt = initial_prompt_1 + initial_prompt_2
separator = "#####"

init_api()
save_history_to_file(initial_prompt_1)

while True :
    prompt = input("You(q:quit): ")
    if prompt=='q':
        break
    sort_history(load_history_from_file(), prompt)
    history = load_history_from_file()
    best_history = get_latest_n_from_history(history, 5)
    full_user_prompt = initial_prompt_2.format(prompt)
    full_prompt = best_history + " \n " + full_user_prompt
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=full_prompt,
        temperature=1,
        max_tokens=100,
        stop=["You:", "AI:"],
    )
    response_text = response.choices[0].text.strip()
    history += " \n " + full_user_prompt + response_text + " \n " + separator + " \n "
    save_history_to_file(history)

    print("AI : " + response_text)

You(q:quit): 안녕


<ipython-input-18-ff9924c1eefe>:37: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return a_without_stopwords.similarity(b_without_stopwords)


AI : 안녕하세요! 무엇을 도와드릴까요?
You(q:quit): 단어 bless의 뜻은 뭐니?
AI : bless의 뜻은 축복하다, 복을 비유적으로 주다 등의 뜻을 갖고 있습니다.
You(q:quit): 단어 korea의 뜻은 뭐니?
AI : 한국은 사회·문화·예술 등 다양한 관점에서 대한민국, 조선민주주의인민공화국, 또는
You(q:quit): 내가 물어본 단어 리스트 알려주렴.
AI : 그래 당신이 물어본 단어 리스트를 알려 드릴게요.
You(q:quit): 뭔데?
AI : 미안해요, 나는 한국어를 모르게 되었습니다. 다른 언어로 묻으시겠습니까?
You(q:quit): q
